## Music store (Chinook database) analysis

### Question 1:
* 1.1 How many customers does the store have in total?
* 1.2 How many invoices (orders) are there in total?
* 1.3 Over how many distinct countries are the customers spread?


In [9]:
#1.1 How many customers does the store have in total?
import pandas as pd

df = pd.read_csv('./data/Chinhook_data/Customer.csv')
df['CustomerId'].count()

np.int64(59)

In [12]:
#1.2 How many invoices (orders) are there in total?
df2 = pd.read_csv('./data/Chinhook_data/Invoice.csv')
df2['InvoiceId'].count()

np.int64(412)

In [18]:
#1.3 Over how many distinct countries are the customers spread?
countries = df2['BillingCountry'].unique()
print(f"There are {len(countries)} unique countries.")

There are 24 unique countries.


### Question 2 – Sales by country and customer
* 2.1a What are the top 5 countries by total sales amount?
* 2.1b For each of those 5 countries, what is the total sales and the number of customers?
* 2.2 For each of those top 5 countries, who is the highest-spending customer (full name) and what is their total spending?

In [27]:
df2.columns

Index(['InvoiceId', 'CustomerId', 'InvoiceDate', 'BillingAddress',
       'BillingCity', 'BillingState', 'BillingCountry', 'BillingPostalCode',
       'Total'],
      dtype='object')

In [54]:
#2.1a What are the top 5 countries by total sales amount?
df = pd.read_csv('./data/Chinhook_data/Invoice.csv')
top5 = (
    df.groupby('BillingCountry')['Total']
      .sum()
      .sort_values(ascending=False)
      .head(5)
      .reset_index(name='Total_expense')   # new column name
)

print(top5)


  BillingCountry  Total_expense
0            USA         523.06
1         Canada         303.96
2         France         195.10
3         Brazil         190.10
4        Germany         156.48


In [ ]:
#2.1b For each of those 5 countries, what is the total sales and the number of customers?
df = pd.read_csv('./data/Chinhook_data/Invoice.csv')

result = (
    df.groupby('BillingCountry')
      .agg(
          Total_expense=('Total', 'sum'),
          num_customers=('CustomerId', 'nunique')
      )
      .sort_values('Total_expense', ascending=False)
      .head(5)
      .reset_index()
)

print(result)


  BillingCountry  Total_expense  num_customers
0            USA         523.06             13
1         Canada         303.96              8
2         France         195.10              5
3         Brazil         190.10              5
4        Germany         156.48              4


In [ ]:
#2.2 For each of those top 5 countries, who is the highest-spending customer (full name) and what is their total spending?

In [31]:
import pandas as pd

df = pd.read_csv('./data/Chinhook_data/Invoice.csv')

df1 = (df.groupby('BillingCountry')
        .agg(Total_Expense=('Total','sum'),
             )
        .sort_values('Total_Expense', ascending=False)
        .head(5)
        )

df2 = df1.merge(df, how='inner', on='BillingCountry')
df2
# 1) Sum Total per (BillingCountry, CustomerId)
df3 = (df2.groupby(['BillingCountry','CustomerId'])
       .agg(total_expense=('Total', 'sum'))
    )
df3
# 2) For each country, keep the customer with highest total_expense
idx = agg.groupby('BillingCountry')['total_expense'].idxmax()
top_customers = agg.loc[idx].reset_index(drop=True)
top_customers

df = pd.read_csv('./data/Chinhook_data/Customer.csv')

df4 = top_customers.merge(df, how='inner', on='CustomerId')
df4 = df4[['CustomerId','FirstName','LastName','BillingCountry','total_expense']]
df4

,CustomerId,FirstName,LastName,BillingCountry,total_expense
0,1,Luís,Gonçalves,Brazil,39.62
1,3,François,Tremblay,Canada,39.62
2,43,Isabelle,Mercier,France,40.62
3,37,Fynn,Zimmermann,Germany,43.62
4,26,Richard,Cunningham,USA,47.62


In [35]:
import pandas as pd

# Load data
invoices = pd.read_csv('./data/Chinhook_data/Invoice.csv')
customers = pd.read_csv('./data/Chinhook_data/Customer.csv')

# (Optional) Top 5 countries by total invoice amount
top_countries = (
    invoices.groupby('BillingCountry', as_index=False)
            .agg(country_expense=('Total', 'sum'))
            .nlargest(5, 'country_expense')
)

# Keep only those countries (remove this block if you want all countries)
invoices_top = invoices.merge(top_countries[['BillingCountry']], on='BillingCountry')

# Total purchase per (country, customer)
country_customer_totals = (
    invoices_top.groupby(['BillingCountry', 'CustomerId'], as_index=False)
                .agg(total_expense=('Total', 'sum'))
)

# For each country, customer with highest total purchase
idx = country_customer_totals.groupby('BillingCountry')['total_expense'].idxmax()
top_customers = country_customer_totals.loc[idx].reset_index(drop=True)

# Attach customer names
result = (
    top_customers
    .merge(customers[['CustomerId', 'FirstName', 'LastName']], on='CustomerId')
    [['CustomerId', 'FirstName', 'LastName', 'BillingCountry', 'total_expense']]
)
result

,CustomerId,FirstName,LastName,BillingCountry,total_expense
0,1,Luís,Gonçalves,Brazil,39.62
1,3,François,Tremblay,Canada,39.62
2,43,Isabelle,Mercier,France,40.62
3,37,Fynn,Zimmermann,Germany,43.62
4,26,Richard,Cunningham,USA,47.62
